# Ok


1. get jira to panda DF format
2. extract features
2.1 bag of words from description, creator, reporter
3. predict priopity
4. predict time estimate/ SP estimate

In [47]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from numpy import *
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('apache.csv', index_col=0, skiprows=0)
df

,projectCategory,created,priority,summary,description,creator,reporter
issue key,,,,,,,
ZOOKEEPER-2769,ZooKeeper,2017-04-28T19:44:50.000+0000,Major,Compiled documentation should not be under sou...,We have xml files that compile into our docume...,abrahamfine,abrahamfine
ZOOKEEPER-2768,ZooKeeper,2017-04-28T09:57:55.000+0000,Critical,Some ideas about four word command,Some ideas about four word command1) about con...,wangzhen,wangzhen
ZOOKEEPER-2767,ZooKeeper,2017-04-27T12:14:48.000+0000,Trivial,Correct the exception messages in X509Util if ...,In org.apache.zookeeper.common.X509Util.org.ap...,a72877,a72877
ZOOKEEPER-2766,ZooKeeper,2017-04-25T10:18:42.000+0000,Major,Quorum fails with java.io.EOFException,When I start a ZooKeeper ensemble comprising 3...,pkleindienst,pkleindienst
ZOOKEEPER-2765,ZooKeeper,2017-04-19T15:43:08.000+0000,Major,modern C++ client,"We should add a modern C++ (i.e. C++14, C++17,...",ecarter,ecarter
ZOOKEEPER-2764,ZooKeeper,2017-04-19T09:54:16.000+0000,Minor,"By default, only srvr four-letter word is on t...",Using the same Vagrant provisioning script as ...,ABchmn,ABchmn
ZOOKEEPER-2763,ZooKeeper,2017-04-19T06:34:02.000+0000,Minor,Utils.toCsvBuffer() omits leading 0 for bytes ...,"org.apache.jute.Utils.toCsvBuffer(), which con...",bberg,bberg
ZOOKEEPER-2762,ZooKeeper,2017-04-18T21:57:58.000+0000,Major,Multithreaded correctness Warnings,NaN,abrahamfine,abrahamfine
ZOOKEEPER-2761,ZooKeeper,2017-04-18T11:31:19.000+0000,Major,Build is packaged as uncompressed tar archive ...,This breaks my build scripts. Did work fine wi...,ABchmn,ABchmn


In [53]:

vect = TfidfVectorizer()
X = vect.fit_transform(df['summary'].values.astype('U')) 
yr = df['priority'].values.astype('U')


encoder = LabelEncoder()
encoder.fit(yr)
y = encoder.transform(yr)


array([2, 1, 4, 2, 2, 3, 3, 2, 2, 0, 2, 4, 3, 2, 2, 2, 2, 2, 3, 2, 2, 3, 2,
       2, 1, 4, 2, 4, 3, 1, 2, 2, 1, 2, 4, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 1, 1, 2, 3, 3, 2, 2, 2, 2, 2, 2, 3,
       3, 2, 2, 1, 2, 2, 2, 0, 2, 3, 3, 1, 2, 2, 2, 2, 0, 2, 3, 2, 2, 2, 2,
       2, 3, 4, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 1, 2, 2, 3,
       2, 2, 2, 2, 2, 4, 2, 0, 2, 2, 3, 2, 0, 3, 2, 1, 2, 2, 0, 0, 2, 3, 4,
       2, 4, 3, 2, 2, 2, 3, 2, 3, 4, 2, 2, 2, 3, 4, 1, 2, 2, 2, 2, 1, 2, 2,
       2, 2, 3, 3, 2, 3, 2, 1, 2, 3, 3, 2, 0, 2, 1, 2, 2, 2, 3, 2, 1, 2, 3,
       2, 2, 3, 2, 3, 2, 0, 3, 2, 2, 2, 2, 1, 2, 3, 3, 3, 3, 3, 2, 2, 2, 2,
       2, 3, 4, 3, 2, 1, 2, 2, 2, 0, 2, 2, 2, 2, 2, 3, 2, 0, 2, 3, 3, 2, 2,
       2, 2, 2, 3, 3, 2, 2, 2, 1, 2, 3, 3, 2, 3, 3, 3, 2, 2, 3, 0, 3, 2, 1,
       1, 2, 2, 2, 2, 2, 3, 3, 3, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 4, 3,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 4, 2, 1, 2, 1, 3, 3, 2, 4, 2, 2,
       3, 1,

In [49]:
TEST_SIZE = 0.2
RANDOM_STATE = 0
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)


In [50]:
y_pred = OneVsOneClassifier(LinearSVC(random_state=0)).fit(X_train, y_train).predict(X_val)
confusion_matrix(y_val, y_pred)

array([[  1,   0,  18,   1,   0],
       [  0,   0,  10,   1,   0],
       [  1,   1, 110,  12,   0],
       [  0,   0,  30,   7,   0],
       [  0,   0,   6,   1,   1]])

In [52]:
y_val

array([2, 2, 2, 4, 3, 2, 4, 2, 1, 2, 2, 2, 0, 2, 2, 0, 4, 1, 2, 2, 2, 2, 2,
       2, 2, 2, 0, 2, 1, 1, 2, 2, 2, 2, 2, 3, 0, 0, 2, 2, 2, 2, 3, 2, 2, 2,
       2, 2, 1, 2, 3, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 4, 2, 3, 2, 1, 2, 2, 1,
       2, 2, 3, 2, 2, 1, 0, 2, 3, 3, 2, 3, 2, 0, 2, 2, 3, 2, 2, 2, 3, 2, 3,
       0, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 3, 4, 2, 2, 0, 2, 2, 2, 2, 0, 2,
       0, 0, 3, 2, 2, 3, 2, 0, 3, 3, 2, 2, 2, 3, 2, 3, 2, 2, 3, 2, 0, 3, 2,
       2, 2, 2, 2, 2, 0, 2, 2, 0, 0, 2, 4, 2, 2, 2, 2, 3, 4, 2, 2, 1, 2, 2,
       3, 3, 2, 1, 2, 1, 2, 2, 4, 2, 2, 2, 2, 3, 2, 3, 3, 0, 3, 3, 2, 3, 2,
       3, 2, 2, 2, 3, 2, 2, 2, 2, 3, 0, 3, 0, 2, 3, 2])

In [8]:
from sklearn import metrics
metrics.accuracy_score(y_val, y_pred)

0.59499999999999997

In [9]:
print(metrics.classification_report(y_val, y_pred))

             precision    recall  f1-score   support

          0       0.50      0.05      0.09        20
          1       0.00      0.00      0.00        11
          2       0.63      0.89      0.74       124
          3       0.32      0.19      0.24        37
          4       1.00      0.12      0.22         8

avg / total       0.54      0.59      0.52       200



In [ ]:
#plot
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.metrics import roc_curve, auc



In [10]:
y[:5]

array([2, 1, 4, 2, 2])

In [11]:
yr[:5]

array([u'Major', u'Critical', u'Trivial', u'Major', u'Major'], 
      dtype='<U8')

In [12]:
encoder.classes_

array([u'Blocker', u'Critical', u'Major', u'Minor', u'Trivial'], 
      dtype='<U8')

In [13]:
X_val

<200x2073 sparse matrix of type '<type 'numpy.float64'>'
	with 1657 stored elements in Compressed Sparse Row format>

In [14]:
def extract(review):
    '''extracts features from review'''
    result = {}
    for word in str(review).split():
        if word in result:
            result[word] += 1
        else:
            result[word] = 1
    return result

In [15]:
reviews = df['description']

In [16]:
X = []
y = []
words = []
# build an index of all occurring words
for review in reviews:
    for word in extract(review):
        if word not in words:
            words.append(word)


In [17]:
words

['and',
 'control',
 'into',
 'them',
 'uncompiled',
 'have',
 'in',
 'our',
 'xml',
 'We',
 'should',
 'source',
 'only',
 'precompiled',
 'under',
 'files',
 'src/docs/',
 'that',
 'docs/We',
 'documentation',
 'remove',
 'compile',
 'data,such',
 'about',
 'word',
 'we',
 'Some',
 'ideas',
 'cons,can',
 'four',
 'add',
 'command',
 'likenow/10.204.2.39:63943[1](queued=0,recved=7,sent=7,sid=0x154c32e8c2a5b8c,lop=PING,est=1483669807748,to=10000,l',
 'command1)',
 'dimension',
 'keystore',
 'exception',
 'locati',
 'for',
 'In',
 'of',
 'contains',
 'messages',
 'related',
 'truststore',
 'org.apache.zookeeper.common.X509Util.org.apache.zookeeper.common.X509Util.createSSLContext',
 'instead',
 'own',
 'is',
 'ZooKeeper',
 'When',
 'quorum,',
 'one',
 'say',
 'out',
 'finally',
 'their',
 'currently',
 'three',
 'start',
 '3',
 '2',
 'nodes',
 'ensemble',
 'node',
 'behavior:Two',
 'I',
 'nodes,',
 "I'm",
 'a',
 'comprising',
 'facing',
 '3)',
 'following',
 'the',
 "(let's",
 'modern',

In [54]:
#next method
df.shape

(1000, 7)

In [55]:
X = df.summary.values.astype('U')
X[:5]

array([u'Compiled documentation should not be under source control',
       u'Some ideas about four word command',
       u'Correct the exception messages in X509Util if truststore location or password is not configured',
       u'Quorum fails with java.io.EOFException', u'modern C++ client'], 
      dtype='<U241')

In [63]:
y =df.priority


In [64]:
y.value_counts().sort_index()

Blocker      70
Critical     67
Major       584
Minor       227
Trivial      52
Name: priority, dtype: int64

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

In [66]:
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

In [67]:
pd.DataFrame(X_train_dtm.toarray(), columns=vect.get_feature_names()).head()

,06,0bytes,10,12,127,14,1576,1653,1663,1910,...,zookeeperserver,zookeeperserverbean,zookeeperserverlistenerimpl,zookeeperservermain,zookeepertest,zookeper,zookkeeper,zoookeeper,zootrace,zxid
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()


In [69]:
nb.fit(X_train_dtm, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [70]:
y_pred_class = nb.predict(X_test_dtm)

In [71]:
metrics.accuracy_score(y_test, y_pred_class)

0.57599999999999996

In [72]:
y_test.value_counts().head(5) / y_test.shape

Major       0.564
Minor       0.248
Blocker     0.068
Trivial     0.060
Critical    0.060
Name: priority, dtype: float64

In [73]:
metrics.confusion_matrix(y_test, y_pred_class)

array([[  2,   0,  14,   1,   0],
       [  0,   0,  13,   2,   0],
       [  0,   0, 134,   7,   0],
       [  0,   0,  54,   8,   0],
       [  0,   0,  11,   4,   0]])

In [74]:
print(metrics.classification_report(y_test, y_pred_class))

             precision    recall  f1-score   support

    Blocker       1.00      0.12      0.21        17
   Critical       0.00      0.00      0.00        15
      Major       0.59      0.95      0.73       141
      Minor       0.36      0.13      0.19        62
    Trivial       0.00      0.00      0.00        15

avg / total       0.49      0.58      0.47       250



In [75]:
#plot
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.metrics import roc_curve, auc

yp = label_binarize(y, classes=[0, 1, 2,3,4])
n_classes = yp.shape[1]

In [37]:
n_classes

5

In [83]:
fpr = dict()
tpr = dict()
roc_auc = dict()

y_test

issue key
ZOOKEEPER-2251    Critical
ZOOKEEPER-1940     Trivial
ZOOKEEPER-2306       Major
ZOOKEEPER-2391       Minor
ZOOKEEPER-2524       Major
ZOOKEEPER-1827       Major
ZOOKEEPER-2504       Major
ZOOKEEPER-1948       Major
ZOOKEEPER-2448       Major
ZOOKEEPER-2720       Major
ZOOKEEPER-2312       Minor
ZOOKEEPER-2627       Major
ZOOKEEPER-1787       Minor
ZOOKEEPER-2414    Critical
ZOOKEEPER-1784       Major
ZOOKEEPER-2636     Blocker
ZOOKEEPER-2665       Major
ZOOKEEPER-2763       Minor
ZOOKEEPER-2158       Major
ZOOKEEPER-2262       Major
ZOOKEEPER-2056       Major
ZOOKEEPER-2286       Minor
ZOOKEEPER-2160     Trivial
ZOOKEEPER-2256       Minor
ZOOKEEPER-2325    Critical
ZOOKEEPER-2650       Major
ZOOKEEPER-2661       Major
ZOOKEEPER-2111       Major
ZOOKEEPER-2169       Major
ZOOKEEPER-2385     Blocker
                    ...   
ZOOKEEPER-2457       Major
ZOOKEEPER-1864       Major
ZOOKEEPER-2465     Blocker
ZOOKEEPER-1836     Trivial
ZOOKEEPER-2164       Major
ZOOKEEPER-1997    

In [82]:
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[i], y_pred_class[ i])
    roc_auc[i] = auc(fpr[i], tpr[i])

TypeError: Singleton array 'Major' cannot be considered a valid collection.